In [ ]:
import sys

In [ ]:
## needed libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd
import numpy as np
import plotly.express as px

### error checking package
import pdb
import re

In [ ]:
with open('2023round2.html', 'r') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
import requests
import csv

In [ ]:
player_draft_analysis = []

# Change this depending on which round you're scraping (1,31 for Round 1) (31, 61 for Round 2)
for i in range(31, 61):
  pick_div = soup.find('div', id=f'pick-{i}')

  if pick_div:
      # Find the <strong>Pre-Draft Analysis</strong> within pick_div
      pre_draft_analysis_h2 = pick_div.find('h2', string='Pre-Draft Analysis')
      pick_div_img = pick_div.find('img')['src']
      athlete_id = pick_div_img.split('/')[-1].split('.')[0]

      print(athlete_id)

      # Change the year in the url depending on the year you're scraping
      api_url = f"https://site.web.api.espn.com/apis/site/v2/sports/basketball/nba/draft/athlete/{athlete_id}?season=2017&region=us&lang=en"

      # Send a GET request to the API endpoint
      response = requests.get(api_url)

      predraft = ''
      postdraft = ''

      data = response.json()
      if 'analysis' in data:
        for analysis in data['analysis']:
          if analysis['type'] == 'predraft':
            predraft = analysis['text']
          if analysis['type'] == 'postdraft':
            postdraft = analysis['text']

        # Find the <strong>Strengths</strong> section
        clean_predraft = BeautifulSoup(predraft, 'html.parser').get_text(separator="\n")
        clean_postdraft = BeautifulSoup(postdraft, 'html.parser').get_text(separator="\n")

        # Find the <strong>Improvement areas</strong> section
        #improvement_areas_section = soup.find('strong', string='Improvement areas').find_next_sibling()

        player_draft_analysis.append({
            'name': data['displayName'],
            'predraft': clean_predraft,
            'postdraft': clean_postdraft
        })

# Define the CSV file path and name
csv_file_path = '2017round2.csv'

# Define the field names (column headers) based on the keys in the dictionary
field_names = ['name', 'predraft', 'postdraft']

# Open the CSV file in write mode
with open(csv_file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.DictWriter(csv_file, fieldnames=field_names)

    # Write the header row
    csv_writer.writeheader()

    # Write each dictionary as a row in the CSV file
    csv_writer.writerows(player_draft_analysis)

print(f"CSV file '{csv_file_path}' created successfully.")



In [ ]:
# SCRAPER FOR 2022 AND 2023, GETTING ONLY MOST RECENT PARAGRAPH

player_draft_analysis = []

# Change this depending on which round you're scraping (1,31 for Round 1) (31, 61 for Round 2)
for i in range(32, 59):
  pick_div = soup.find('div', id=f'pick-{i}')

  if pick_div:
      # Find the <strong>Pre-Draft Analysis</strong> within pick_div
      pre_draft_analysis_h2 = pick_div.find('h2', string='Pre-Draft Analysis')
      pick_div_img = pick_div.find('img')['src']
      athlete_id = pick_div_img.split('/')[-1].split('.')[0]

      print(i, athlete_id)



      # These were only for the 2022 draft picks that didn't have a picture
      if athlete_id == "nophoto":
        if i == 55:
          athlete_id = 105830
        elif i == 56:
          athlete_id = 106302

      # Change the year in the url depending on the year you're scraping

      api_url = f"https://site.web.api.espn.com/apis/site/v2/sports/basketball/nba/draft/athlete/{athlete_id}?season=2022&region=us&lang=en"

      # Send a GET request to the API endpoint
      response = requests.get(api_url)

      most_recent_predraft = ""
      postdraft = ''

      data = response.json()
      predraft_analysis = []
      if 'analysis' in data:
        for analysis in data['analysis']:
          if analysis['type'] == 'predraft' and len(analysis['text']) > 0:
            predraft = analysis['text']


            curr_element = pick_div.find('b')
            while curr_element.next_sibling:
              curr_element = curr_element.next_sibling
              if curr_element.name == "b":
                  break
              else:
                  predraft_analysis.append(str(curr_element))

      predraft_analysis = "".join(predraft_analysis)

      clean_predraft = BeautifulSoup(predraft_analysis, 'html.parser').get_text(separator="\n")

      if 'code' in data:
        continue

      player_draft_analysis.append({
          'name': data['displayName'] ,
          'predraft': clean_predraft,
      })

# Define the CSV file path and name
csv_file_path = '2023round2test.csv'

# Define the field names (column headers) based on the keys in the dictionary
field_names = ['name', 'predraft']

# Open the CSV file in write mode
with open(csv_file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.DictWriter(csv_file, fieldnames=field_names)

    # Write the header row
    csv_writer.writeheader()

    # Write each dictionary as a row in the CSV file
    csv_writer.writerows(player_draft_analysis)

print(f"CSV file '{csv_file_path}' created successfully.")

